In [ ]:
# Install the latest version
!pip install zai-sdk
# Or specify a version
!pip install zai-sdk==0.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.10.1
    Uninstalling PyJWT-2.10.1:
      Successfully uninstalled PyJWT-2.10.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: zai-sdk
    Found existing installation: zai-sdk 0.0.3.3
    Uninstalling zai-sdk-0.0.3.3:
      Successfully uninstalled zai-sdk-0.0.3.3


In [ ]:
import zai
print(zai.__version__)

0.0.3


In [ ]:
import os, io, base64, time, random, pathlib
from typing import Iterator
from PIL import Image
from openai import OpenAI, RateLimitError, APIError#, ServiceUnavailableError
from zai import ZaiClient
from google.colab import drive
drive.mount('/content/drive')

def iter_image_paths(root_dir: str, exts={".jpg",".jpeg",".png",".webp",".bmp"}) -> Iterator[pathlib.Path]:
    root = pathlib.Path(root_dir)
    for p in root.rglob("*"):
        if p.is_file() and p.suffix.lower() in exts:
            yield p


In [ ]:
PROMPT = """You are a visual verifier for computer vision detections.
TASK
Given a satellite image with a bounding box labeled "bus-stop", decide if the detection is:
- correct : it is indeed a bus stop
- incorrect : it is not a bus stop
POSITIVE VISUAL CUES
- Small shelter or roofed structure next to the road (rectangular shadow is common).
- Vertical post/sign directly by the roadside.
- Bay, widened shoulder, or compacted area where vehicles can stop.
- Bench, platform, or rectangular object aligned with the road edge.
NEGATIVE VISUAL CUES (common mislabels)
- Ditch, culvert, driveway, or rural entrance without shelter/sign.
- Tree, bush, rock, shadow, or random vegetation patch.
- Object far from the road or clearly not related to transport.
OUTPUT FORMAT (strict, one line per image)
<verdict> : <brief specific reason>
EXAMPLES
- correct : small roofed shelter with clear roadside bay.
- incorrect : shadow of a tree, no bus stop structure.
- incorrect : field entrance without sign or shelter.
RESPONSE INSTRUCTIONS
- Always output exactly “correct” or “incorrect” followed by a colon and short reason.
- Be concise (max 20 words).
- Do not include the model’s original label or probability."""

client = ZaiClient(api_key="f22d5d7d0f3746c88f40ed9cffa6508a.dLCSbP7UICmLPR0r")  # Enter your own APIKey


In [ ]:
paradas_seguras = "/content/drive/MyDrive/paradas_seguras"

In [ ]:
import base64, glob, os

def to_data_uri(path):
    mime = "image/png" if path.lower().endswith(".png") else "image/jpeg"
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode()
    return f"data:{mime};base64,{b64}"

def process_image(path):
    response = client.chat.completions.create(
        model="glm-4.5v",
        messages=[{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": to_data_uri(path)}},
                {"type": "text",  "text": PROMPT}
            ]
        }],
        thinking={"type": "enabled"}
    )
  #  print(response.choices[0].message)
    return response.choices[0].message

In [ ]:
import os, shutil
from IPython.display import display, Image as IPImage

# carpetas destino
REAL = "/content/drive/MyDrive/paradas_reales"
REV  = "/content/drive/MyDrive/paradas_a_revisar"
os.makedirs(REAL, exist_ok=True)
os.makedirs(REV,  exist_ok=True)

for img in glob.glob(os.path.join(paradas_seguras, "*.[pjP][pnN][gG]")):
    display(IPImage(filename=img))
    procesado = process_image(img)          # tu función
    veredicto = "correct" if ">correct" in procesado.content else "incorrect"

    destino = REAL if veredicto == "correct" else REV
    shutil.copy(img, os.path.join(destino, os.path.basename(img)))
    print(f"{veredicto} → guardada en {destino}")


Criterio para descartar o ubicar parada

*   ambos modelos dicen lo mismo, lo dejamos como cierto;
*   los modelos difieren entonces, lo clasificamos nosotros.



In [ ]:
paradas_no_seguras="/content/drive/MyDrive/paradas_no_seguras"

In [ ]:
len(glob.glob(os.path.join(paradas_no_seguras, "*.[pjP][pnN][gG]")))

754

In [ ]:
for img in glob.glob(os.path.join(paradas_no_seguras, "*.[pjP][pnN][gG]")):
  print(img)

/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11285_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11310_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11316_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11359_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11366_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11420_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11435_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11460_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11470_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11481_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11489_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11517_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11566_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11629_z20.png
/content/drive/MyDrive/paradas_no_seguras/ruta8_pt_11693_z20.png
/content/drive/MyDrive/pa

In [ ]:
# carpetas destino
NO_REAL = "/content/drive/MyDrive/paradas_no_reales"
os.makedirs(NO_REAL, exist_ok=True)
i=1
for img in glob.glob(os.path.join(paradas_no_seguras, "*.[pjP][pnN][gG]")):
    display(IPImage(filename=img))
    procesado = process_image(img)          # tu función
    veredicto = "correct" if ">correct" in procesado.content else "incorrect"

    destino = REV if veredicto == "correct" else NO_REAL
    shutil.copy(img, os.path.join(destino, os.path.basename(img)))
    print(f"{veredicto} → guardada en {destino}")
    print("faltan:", 754-i)
    i+= 1